In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import asyncio
import datetime
from datetime import datetime
from datetime import timedelta, timezone
from typing import Optional
from dotenv import load_dotenv
from operator import itemgetter

load_dotenv()
PWD = os.getenv("PWD")
db_name = PWD + "\\database" + "\\RVNUSDT.db"
import sys

sys.path.insert(1, PWD + "\\modules")
from alg_modules.alg_handler import AlgHandler
from plot_modules.candle_plot import CandlePlot
from collections import deque
from paper_trade import PaperTrader
import time
import logging

DEBUG = __debug__
LOG_FILE_NAME = "log_file_name.log"
format = "%(asctime)s [%(levelname)s]: %(message)s"
logger = logging.basicConfig(
    filename=LOG_FILE_NAME if not DEBUG else None,
    format=format,
    encoding="utf-8",
    level=logging.INFO,
)
if not DEBUG:
    logging.getLogger(logger).addHandler(logging.StreamHandler())

from stop_loss import StopLoss
from trade_strategy import TradeStrategy
from wss_thread import WssThread
from api_modules.open_binance_api import OpenBinanceApi
import pytz
tzdata = pytz.timezone('Europe/Moscow') 

In [2]:
def compute_timedelta(dt: datetime):
    if dt.tzinfo is None:
        dt = dt.astimezone()
    now = datetime.now(timezone.utc)
    return max((dt - now).total_seconds(), 0)
async def sleep_until(when: datetime, result = None):
    """|coro|
    Sleep until a specified time.
    If the time supplied is in the past this function will yield instantly.
    .. versionadded:: 1.3
    Parameters
    -----------
    when: :class:`datetime.datetime`
        The timestamp in which to sleep until. If the datetime is naive then
        it is assumed to be local time.
    result: Any
        If provided is returned to the caller when the coroutine completes.
    """
    delta = compute_timedelta(when)
    return await asyncio.sleep(delta, result)
# def get_data():
#     df = OpenBinanceApi.get_df(
#         pair = 'RVNUSDT',
#         interval = '1m',
#         limit = 1,
#     )
#     return df

    

In [3]:
server_time = datetime.fromtimestamp(OpenBinanceApi.server_time()/1000)
local_time = datetime.now()
delay = server_time - local_time
logging.info(f'server_time: {server_time}')
# logging.info(f'local_time: {local_time}')
delay.total_seconds()

2022-02-26 12:10:30,765 [INFO]: server_time: 2022-02-26 12:10:48.010000


17.244564

In [4]:

# try:
#     while True:

#         #region
#         # for _, row in df.iterrows():
#         # if 1: # wait for new data comes in
#         #     window.append(dict(row))
#         #     df_ = pd.DataFrame(window)
#         #     alg.update_data(df_)
#         #     alg.calculate(val_col='open_', time_col='open_time',)
#         #     do_trade, cross_type = alg.evaluate()
#         #     if STOP_LOSS_ENABLED:
#         #         stop_loss_trade_flag, trade_data = stop_loss.stop_loss_alg(
#         #                                                 trade_data=trade_data,
#         #                                                 p_trdr=p_trdr,
#         #                                                 row=row,
#         #                                             )

#         #     if do_trade:
#         #         trade_data, stop_loss_trade_flag = TradeStrategy.trade_alg(stop_loss_trade_flag, trade_data, p_trdr, row, cross_type)
#         # if w.get_data() is not None:
#         #     print(itemgetter('a', 'b', 'c')(w.get_data()))

#         # time.sleep(1)
#         #endregion
# finally:
#     # close thread
#     w.close()
    

In [5]:
# notifications
from win10toast import ToastNotifier
toast = ToastNotifier()
static_notification_settings = dict(
    title="Algo traid BOT",
    duration = 20,
    icon_path = "python.ico",
    threaded = 1,
)

notify = lambda msg: toast.show_toast(
    msg=msg,
    **static_notification_settings,
    )

msg="Watch out for notifications from here"
async def notification(msg):
    if notify(msg):
        pass
    else:
        await asyncio.sleep(20)
        notify(msg)

await notification(msg)

In [6]:
DATA_AWAIT_TIME = 1 # seconds
SERVER_DELAY = 10 # seconds
INTERVAL_SECONDS = 60 # seconds
w = WssThread(
    url='wss://stream.binance.com:9443/ws/rvnusdt@ticker',
    maxlen=10,
    )
w.start()

STOP_LOSS_ENABLED=True
STOP_LOSS_THRESHOLD=-1.3

DEQUE_MAX_LENGTH = 200
INTERVAL = '1m'


df = OpenBinanceApi.get_df(
    pair = 'RVNUSDT',
    interval = INTERVAL,
    limit = 1000,
    )
# drop last row TODO make assert to not dublicate last row from cycle
df = df[:-1]
stop_loss_trade_flag = False
MA_list = (2, 25, 100, 200)

window = deque(maxlen=200)
for i, row in df.iterrows():
    window.append(dict(row.squeeze()))
#initial currency resources
p_trdr = PaperTrader(
    main_currency_label='RVN',
    secondary_currency_label='USD',
    main_currency_amount=100,
    secondary_currency_amount=0,
    fee=0.1,
)
trade_data = pd.DataFrame(
    columns = p_trdr.get_df(timestamp=df.iloc[-1]['Date']).columns.values
)
stop_loss = StopLoss(
    STOP_LOSS_THRESHOLD=STOP_LOSS_THRESHOLD,
)
# init alg
alg = AlgHandler(
    df=pd.DataFrame([]),
    MA_list=MA_list,
    )
try:
    while 1:
        logging.info('===get new data===')
        new_df = OpenBinanceApi.get_df(
                pair = 'RVNUSDT',
                interval = INTERVAL,
                limit = 1,
            )
        dt = datetime.fromtimestamp(int(new_df.Real_Date[-1:])/1000)
        server_time = datetime.fromtimestamp(OpenBinanceApi.server_time()/1000)
        logging.debug(f'server time: {server_time}   {server_time.minute=}, {dt.minute=}')
        # extract function?

        if server_time.minute == dt.minute:
            logging.debug('+++===success===+++')
            window.append(dict(new_df.squeeze()))
            df_ = pd.DataFrame(window)
            # === process data here ===
            # display(df_)
            for _, row in df.iterrows():
                window.append(dict(row))
                df__ = pd.DataFrame(window)
                alg.update_data(df__)
                alg.calculate(val_col='Open', time_col='Date',)
                do_trade, cross_type = alg.evaluate()
                if STOP_LOSS_ENABLED:
                    stop_loss_trade_flag, trade_data = stop_loss.stop_loss_alg(
                                                            trade_data=trade_data,
                                                            p_trdr=p_trdr,
                                                            row=row,
                                                        )

            if do_trade:
                trade_data, stop_loss_trade_flag = TradeStrategy.trade_alg(stop_loss_trade_flag, trade_data, p_trdr, row, cross_type)
                await notification(f'Trade done: {trade_data}\n {trade_data[-1:]}')
            logging.info(f'do trade: {do_trade}')
            display(trade_data)
            display(alg.crosses)
            # === end of data processing ===
            time_to_sleep = dt - delay + timedelta(seconds=SERVER_DELAY) + timedelta(seconds=INTERVAL_SECONDS)
            server_delay = dt - server_time
            logging.debug(f'server valid time: {server_time}')
            logging.debug(f'server delay: {server_delay.total_seconds()}')
            logging.debug(f'sleep till: {time_to_sleep}')
            await sleep_until(time_to_sleep)
        else:
            logging.debug('---not valid---')
            logging.debug('sleep 1 sec')
            await asyncio.sleep(DATA_AWAIT_TIME)
        
finally:
    # close thread
    w.close()
    # trade.close()
    # notify that we done
    await notification('App terminated')


2022-02-26 12:10:31,919 [INFO]: ===get new data===
2022-02-26 12:10:32,097 [INFO]: ### Opening wss stream ###
2022-02-26 12:10:36,450 [INFO]: do trade: False


,type,sell_price,buy_price,fee,RVN,USD,_amount,timestamp


[MA_point(timestamp=Timestamp('2022-02-26 10:59:00'), val=0.059254799999999996, type='raise'),
 MA_point(timestamp=Timestamp('2022-02-26 11:50:00'), val=0.0592772, type='fall')]

2022-02-26 12:10:52,763 [INFO]: ===get new data===
2022-02-26 12:10:56,960 [INFO]: do trade: False


,type,sell_price,buy_price,fee,RVN,USD,_amount,timestamp


...

,type,sell_price,buy_price,fee,RVN,USD,_amount,timestamp


[MA_point(timestamp=Timestamp('2022-02-26 10:59:00'), val=0.059254799999999996, type='raise'),
 MA_point(timestamp=Timestamp('2022-02-26 11:50:00'), val=0.0592772, type='fall')]

2022-02-26 16:19:52,762 [INFO]: ===get new data===
2022-02-26 16:19:57,072 [INFO]: do trade: False


,type,sell_price,buy_price,fee,RVN,USD,_amount,timestamp


[MA_point(timestamp=Timestamp('2022-02-26 10:59:00'), val=0.059254799999999996, type='raise'),
 MA_point(timestamp=Timestamp('2022-02-26 11:50:00'), val=0.0592772, type='fall')]

In [ ]:
assert 0

In [ ]:
trade_data

NameError: name 'trade_data' is not defined

In [ ]:
for i, row in df.iterrows():
    window.append(dict(row.squeeze()))

In [ ]:
pd.DataFrame(window)

,Date,Open,High,Low,Close,Real_Date
0,2022-02-23 16:43:00,0.05698000,0.05704000,0.05698000,0.05704000,1645623780000
1,2022-02-23 16:44:00,0.05703000,0.05704000,0.05696000,0.05696000,1645623840000
2,2022-02-23 16:45:00,0.05697000,0.05702000,0.05697000,0.05698000,1645623900000
3,2022-02-23 16:46:00,0.05697000,0.05699000,0.05693000,0.05693000,1645623960000
4,2022-02-23 16:47:00,0.05694000,0.05700000,0.05694000,0.05694000,1645624020000
...,...,...,...,...,...,...
195,2022-02-23 19:58:00,0.05692000,0.05694000,0.05690000,0.05690000,1645635480000
196,2022-02-23 19:59:00,0.05686000,0.05691000,0.05686000,0.05691000,1645635540000
197,2022-02-23 20:00:00,0.05694000,0.05695000,0.05686000,0.05686000,1645635600000
198,2022-02-23 20:01:00,0.05689000,0.05695000,0.05679000,0.05679000,1645635660000


In [ ]:
df = df[:-1]
df

,Date,Open,High,Low,Close,Real_Date
0,2022-02-23 03:31:00,0.05501000,0.05505000,0.05501000,0.05502000,1645576260000
1,2022-02-23 03:32:00,0.05498000,0.05498000,0.05475000,0.05476000,1645576320000
2,2022-02-23 03:33:00,0.05480000,0.05480000,0.05463000,0.05472000,1645576380000
3,2022-02-23 03:34:00,0.05469000,0.05473000,0.05463000,0.05473000,1645576440000
4,2022-02-23 03:35:00,0.05475000,0.05477000,0.05466000,0.05466000,1645576500000
...,...,...,...,...,...,...
994,2022-02-23 20:05:00,0.05668000,0.05668000,0.05650000,0.05659000,1645635900000
995,2022-02-23 20:06:00,0.05658000,0.05658000,0.05651000,0.05655000,1645635960000
996,2022-02-23 20:07:00,0.05655000,0.05659000,0.05655000,0.05659000,1645636020000
997,2022-02-23 20:08:00,0.05659000,0.05664000,0.05659000,0.05662000,1645636080000


In [ ]:
def process_data(new_df: pd.DataFrame, window: deque,) -> pd.DataFrame:
    window.append(new_df)
    df_ = pd.DataFrame(window)
    return df_

In [ ]:
dict(new_df.squeeze())

NameError: name 'new_df' is not defined

In [ ]:
window[0]

In [ ]:
trade_data